In [5]:
import pandas as pd
import os
import cv2
import numpy as np
import json

In [125]:
def Detect_Office(Json,Office):

    NewList=Json['fields']
    Dict=list()
    for d in NewList:
        try:
            newDict={}
            newDict['inferText']=d['inferText']
            newDict['boundingPoly']=d['boundingPoly']
            Dict.append(newDict)
        except KeyError:
            continue

    res = [d
       for d in Dict 
       if (Office[0] == d['inferText']) or (Office == d['inferText']) or (Office[-2:][0] == d['inferText'])]

    if len(res)!=0:
        res = res[0]['boundingPoly']['vertices']
        Edge=max(int(d['x']) for d in res)
        return(Edge)
    else:
        return(None)

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

### CLOVA FUNCTION ###
import requests
import uuid
import time
import json
import cv2
import base64

api_url = 'https://deelieyxuc.apigw.ntruss.com/custom/v1/1972/ebd01bcbf693d069817622e9839e20914143c7d0d8953eddee40e8b0af96c95b/general'
secret_key = 'S1NmVXpYZlJ0cGJ0ZEFRZXVlbkRkaHFReE9FcHNTQ0U='

#ここ何やってるか知りたい↓ここで全てのページをいったん画像→OCRにしてる？
def Clova(Year,Page):
    path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
    with open(path+"Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg",'rb') as f:
         file_data = f.read()

    request_json = {
            'images': [
                {
                    'format': 'jpg',
                    'name': 'demo',
                    'data':base64.b64encode(file_data).decode()}],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000)),
            'lang':'ja'
            }
    payload = json.dumps(request_json).encode("UTF-8")
    headers = {'X-OCR-SECRET': secret_key,
              'Content-Type': 'application/json'}
    response = requests.request("POST", api_url, headers=headers, data = payload)
    Json=json.loads(response.text)['images'][0]
    
    return Json

In [7]:
#Research Notes (keitaro2@illinois.edu)ないけど、どんなファイル入ってる?
#json.loadsがOCRで読みだすやつってこと？

Year='1934'
Showa='9'
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
os.chdir(path)
df = pd.read_csv('C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Processed_Data/Index/S'+Showa+'.csv')
df=df.drop(df.columns[0], axis=1)

#file_pathで
file_path='C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\DataFrame.json'
with open(file_path, encoding="utf-8") as f:
    dta = json.loads(f.read())

In [8]:
dta

{'1934': {'Admin': {'文書課': {'Starting_Page': 3,
    'Office_X1': 292,
    'Ending_Page': 5,
    'Office_X2': 439,
    'Page_Range': [3, 4, 5],
    'Position': {'Manager': {'XLocation': 261,
      'Page': 3,
      'EndLocation': 230.0,
      'EndPage': 3.0},
     'Leader': {'XLocation': 230,
      'Page': 3,
      'EndLocation': 469.0,
      'EndPage': 4.0},
     'Admin': {'XLocation': 469,
      'Page': 4,
      'EndLocation': 302.0,
      'EndPage': 4.0,
      'Data': [{'Name': '淺野元三郎', 'Wage': '幸'},
       {'Name': '五人店TE', 'Wage': '月七'},
       {'Name': '小川有道', 'Wage': '𠮷'},
       {'Name': '志賀漆', 'Wage': 'お'},
       {'Name': '京山重長', 'Wage': '1'},
       {'Name': '天主任人江博房', 'Wage': '万九和'},
       {'Name': '各个長下條装', 'Wage': 'カト'},
       {'Name': '和久田賞', 'Wage': '上'},
       {'Name': '木村五', 'Wage': 'あや'},
       {'Name': '長坂番', 'Wage': '幸'},
       {'Name': '宮尾D20', 'Wage': '吉'},
       {'Name': '正版 松留', 'Wage': '七七'},
       {'Name': '鍜田國信', 'Wage': '七上'},
       {'Name': '部見安二郎', 

In [9]:
#Test code| Version 2#
#Show Working office#
n=2

#Extract key info of office
Row  = df.iloc[n]
ExRow= df.iloc[n-1]
Row,ExRow

(Office      文書課
 Dept      Admin
 Year          9
 Page          3
 Name: 2, dtype: object,
 Office      職員課
 Dept      Admin
 Year          9
 Page          2
 Name: 1, dtype: object)

In [10]:
#
Showa=ExRow["Year"]

Page=Row["Page"]
ExPage=ExRow["Page"]

Dept=Row["Dept"]
ExDept=ExRow["Dept"]

Office=Row["Office"]
ExOffice=ExRow["Office"]

dta[str(Year)][Dept][Office]={}
dta[str(Year)][ExDept][ExOffice]={}
dta[str(Year)][ExDept][ExOffice]["Starting_Page"]=ExPage

In [11]:
#Test code| Version 2#
#Decompose info into elements
Page=int(Row["Page"])
Office=Row["Office"]
print(Office)
ExPage=int(ExRow["Page"])
ExOffice=ExRow["Office"]


###Insert Starting page information to motherframe###
dta[Year][Dept][Office]={}
dta[Year][Dept][Office]["Starting_Page"]=Page
print(dta)

###Collect Location information###
##Read image for first page##
img=cv2.imread("Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg")
#Convert to json via CLOVA
Json=Clova(Year,Page)

#Find X coordinate of 'Office'.
XCoord_Unit=Detect_Office(Json,Office)
#Add to motherframe
dta[str(Year)][Dept][Office]["Office_X1"]=XCoord_Unit
dta[str(Year)][ExDept][ExOffice]["Ending_Page"]=Page
dta[str(Year)][ExDept][ExOffice]["Office_X2"]=XCoord_Unit+10
dta[str(Year)][ExDept][ExOffice]["Page_Range"]=list(range(ExPage,Page+1))
dta[str(Year)][Dept]

文書課
{'1934': {'Admin': {'文書課': {'Starting_Page': 3}, '職員課': {'Starting_Page': 2}, '庶務課': {'Ending_Page': 5, 'Office_X2': 160, 'Page_Range': [5]}, '監察課': {'Starting_Page': 5, 'Office_X1': 150}}, '中央卸売市場': {}, '保健局': {}, '土木局': {}, '教育局': {}, '水道局': {}, '港湾部（S8.11.1）': {}, '産業局': {}, '監査局': {'庶務課': {'Starting_Page': 5, 'Office_X1': 429}, '監察課': {'Starting_Page': 5, 'Office_X1': 150}}, '社会局': {}, '財務局': {}, '電気局(S8.11.6)': {}, '電気研究所（S8.11.18）': {}}}


{'文書課': {'Starting_Page': 3, 'Office_X1': 292},
 '職員課': {'Starting_Page': 2,
  'Ending_Page': 3,
  'Office_X2': 302,
  'Page_Range': [2, 3]},
 '庶務課': {'Ending_Page': 5, 'Office_X2': 160, 'Page_Range': [5]},
 '監察課': {'Starting_Page': 5, 'Office_X1': 150}}

In [17]:
#Test code| Version 2#
#Show Working office#
n=3

#Extract key info of office
Row  = df.iloc[n]
ExRow= df.iloc[n-1]
Row

Office    庶務課
Dept      監査局
Year        9
Page        5
Name: 3, dtype: object

In [18]:
Page=int(Row["Page"])
Dept=Row["Dept"]
Office=Row["Office"]
print(Office)
ExPage=int(ExRow["Page"])
ExDept=ExRow["Dept"]
ExOffice=ExRow["Office"]


###Insert Starting page information to motherframe###
dta[Year][Dept][Office]={}
dta[Year][Dept][Office]["Starting_Page"]=Page
print(dta)

###Collect Location information###
##Read image for first page##
img=cv2.imread("Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg")
#Convert to json via CLOVA
Json=Clova(Year,Page)

#Find X coordinate of 'Office'.
XCoord_Unit=Detect_Office(Json,Office)
#Add to motherframe
dta[str(Year)][Dept][Office]["Office_X1"]=XCoord_Unit
dta[str(Year)][ExDept][ExOffice]["Ending_Page"]=Page
dta[str(Year)][ExDept][ExOffice]["Office_X2"]=XCoord_Unit+10
dta[str(Year)][ExDept][ExOffice]["Page_Range"]=list(range(ExPage,Page+1))
dta

庶務課
{'1934': {'Admin': {'文書課': {'Starting_Page': 3, 'Office_X1': 292, 'Ending_Page': 5, 'Office_X2': 439, 'Page_Range': [3, 4, 5]}, '職員課': {'Starting_Page': 2, 'Ending_Page': 3, 'Office_X2': 302, 'Page_Range': [2, 3]}, '庶務課': {'Ending_Page': 5, 'Office_X2': 160, 'Page_Range': [5]}, '監察課': {'Starting_Page': 5, 'Office_X1': 150}}, '中央卸売市場': {}, '保健局': {}, '土木局': {}, '教育局': {}, '水道局': {}, '港湾部（S8.11.1）': {}, '産業局': {}, '監査局': {'庶務課': {'Starting_Page': 5}, '監察課': {'Starting_Page': 5, 'Office_X1': 150}}, '社会局': {}, '財務局': {}, '電気局(S8.11.6)': {}, '電気研究所（S8.11.18）': {}}}


{'1934': {'Admin': {'文書課': {'Starting_Page': 3,
    'Office_X1': 292,
    'Ending_Page': 5,
    'Office_X2': 439,
    'Page_Range': [3, 4, 5]},
   '職員課': {'Starting_Page': 2,
    'Ending_Page': 3,
    'Office_X2': 302,
    'Page_Range': [2, 3]},
   '庶務課': {'Ending_Page': 5, 'Office_X2': 160, 'Page_Range': [5]},
   '監察課': {'Starting_Page': 5, 'Office_X1': 150}},
  '中央卸売市場': {},
  '保健局': {},
  '土木局': {},
  '教育局': {},
  '水道局': {},
  '港湾部（S8.11.1）': {},
  '産業局': {},
  '監査局': {'庶務課': {'Starting_Page': 5, 'Office_X1': 429},
   '監察課': {'Starting_Page': 5, 'Office_X1': 150}},
  '社会局': {},
  '財務局': {},
  '電気局(S8.11.6)': {},
  '電気研究所（S8.11.18）': {}}}

In [19]:
#Test code| Version 2#
#Show Working office#
n=4

#Extract key info of office
Row  = df.iloc[n]
ExRow= df.iloc[n-1]
Row

Office    監察課
Dept      監査局
Year        9
Page        5
Name: 4, dtype: object

In [20]:
Page=int(Row["Page"])
Office=Row["Office"]
print(Office)
ExPage=int(ExRow["Page"])
ExOffice=ExRow["Office"]


###Insert Starting page information to motherframe###
dta[Year][Dept][Office]={}
dta[Year][Dept][Office]["Starting_Page"]=Page
print(dta)

###Collect Location information###
##Read image for first page##
img=cv2.imread("Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg")
#Convert to json via CLOVA
Json=Clova(Year,Page)

#Find X coordinate of 'Office'.
XCoord_Unit=Detect_Office(Json,Office)
#Add to motherframe
dta[str(Year)][Dept][Office]["Office_X1"]=XCoord_Unit
dta[str(Year)][ExDept][ExOffice]["Ending_Page"]=Page
dta[str(Year)][ExDept][ExOffice]["Office_X2"]=XCoord_Unit+10
dta[str(Year)][ExDept][ExOffice]["Page_Range"]=list(range(ExPage,Page+1))
dta

監察課
{'1934': {'Admin': {'文書課': {'Starting_Page': 3, 'Office_X1': 292, 'Ending_Page': 5, 'Office_X2': 439, 'Page_Range': [3, 4, 5]}, '職員課': {'Starting_Page': 2, 'Ending_Page': 3, 'Office_X2': 302, 'Page_Range': [2, 3]}, '庶務課': {'Ending_Page': 5, 'Office_X2': 160, 'Page_Range': [5]}, '監察課': {'Starting_Page': 5, 'Office_X1': 150}}, '中央卸売市場': {}, '保健局': {}, '土木局': {}, '教育局': {}, '水道局': {}, '港湾部（S8.11.1）': {}, '産業局': {}, '監査局': {'庶務課': {'Starting_Page': 5, 'Office_X1': 429}, '監察課': {'Starting_Page': 5}}, '社会局': {}, '財務局': {}, '電気局(S8.11.6)': {}, '電気研究所（S8.11.18）': {}}}


{'1934': {'Admin': {'文書課': {'Starting_Page': 3,
    'Office_X1': 292,
    'Ending_Page': 5,
    'Office_X2': 439,
    'Page_Range': [3, 4, 5]},
   '職員課': {'Starting_Page': 2,
    'Ending_Page': 3,
    'Office_X2': 302,
    'Page_Range': [2, 3]},
   '庶務課': {'Ending_Page': 5, 'Office_X2': 160, 'Page_Range': [5]},
   '監察課': {'Starting_Page': 5, 'Office_X1': 150}},
  '中央卸売市場': {},
  '保健局': {},
  '土木局': {},
  '教育局': {},
  '水道局': {},
  '港湾部（S8.11.1）': {},
  '産業局': {},
  '監査局': {'庶務課': {'Starting_Page': 5, 'Office_X1': 429},
   '監察課': {'Starting_Page': 5, 'Office_X1': 150}},
  '社会局': {},
  '財務局': {},
  '電気局(S8.11.6)': {},
  '電気研究所（S8.11.18）': {}}}

In [132]:
#Test code| Version 2#
#Show Working office#
FailedList=[]
for n in range(2,len(df)):
    #Extract key info of office
    Row  = df.iloc[n]
    ExRow= df.iloc[n-1]

    Page=int(Row["Page"])
    Office=Row["Office"]
    Dept=Row['Dept']

    ExPage=int(ExRow["Page"])
    ExOffice=ExRow["Office"]
    ExDept=ExRow['Dept']

    ###Insert Starting page information to motherframe###
    dta[Year][Dept][Office]={}
    dta[Year][Dept][Office]["Starting_Page"]=Page

    ###Collect Location information###
    ##Read image for first page##
    img=cv2.imread("Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg")
    #Convert to json via CLOVA
    Json=Clova(Year,Page)

    #Find X coordinate of 'Office'.
    XCoord_Unit=Detect_Office(Json,Office)
    if XCoord_Unit==None:
        #Add to motherframe
        dta[str(Year)][Dept][Office]["Office_X1"]='NA'
        dta[str(Year)][ExDept][ExOffice]["Ending_Page"]='NA'
        dta[str(Year)][ExDept][ExOffice]["Office_X2"]='NA'
        dta[str(Year)][ExDept][ExOffice]["Page_Range"]='NA'
        print(Office+ 'failed')
        FailedList.append(list((Dept,Office)))
        continue
    else:
        dta[str(Year)][Dept][Office]["Office_X1"]=XCoord_Unit
        dta[str(Year)][ExDept][ExOffice]["Ending_Page"]=Page
        dta[str(Year)][ExDept][ExOffice]["Office_X2"]=XCoord_Unit+10
        dta[str(Year)][ExDept][ExOffice]["Page_Range"]=list(range(ExPage,Page+1))       
        HH=img.shape[:2][0]
        print(Office+ 'success')
        cv2.line(img, (XCoord_Unit,0), (XCoord_Unit,HH), (255,0,0), 2)
        cv2.imshow('pic',img)
        cv2.waitKey(0)



文書課success
庶務課success
監察課success
区政課success
統計課success
都市計画課failed
主計課success
公債課（S9.2.8）success
収納課success
経理課success
地理課success
会計課success
庶務課success
商工課success
農魚課success
権度課success
庶務課success
学務課success
社会教育課failed
体育課success
視学課success
庶務課success
保護課success
福利課success
職業課success
庶務課success
衛生課success
清掃課success
公園課success
庶務課success
会計課success
業務課failed
給水課failed
拡張課failed
庶務課success
道路管理課success
道路建設課failed
河川課failed
下水課success
建築課success
局長室success
庶務課success
労働課success
会計課success
電車課success
自動車課success
電灯課success
電力課success
工務課success
病院success
庶務課success
技術課success
港務所success
庶務課success
企画課success


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\1934\\Page140\\Page140.jpg'

In [133]:
if len(FailedList)/len(df)<0.2:
    print('Fantastic!! Success Rate is '+str(1-(len(FailedList)/len(df))))
else:
    print('残念...Success Rate is '+str(1-(len(FailedList)/len(df))))

Fantastic!! Success Rate is 0.8833333333333333


In [128]:
#Fixing Failed Offices
#Step1: Check for simple page assignment error
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
for n in range(2,len(FailedList)):
    Dept=FailedList[n][0]
    Office=FailedList[n][1]
    print(Dept,Office)
    Page=df['Page'][(df['Office']==Office) & (df['Dept']==Dept)].tolist()[0]
    image=cv2.imread(path+"Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg")
    cv2.imshow('Image',image)
    cv2.waitKey(0)

水道局 業務課
水道局 給水課
水道局 拡張課
土木局 道路建設課
土木局 河川課


In [121]:
n=10
Dept=FailedList[n][0]
Office=FailedList[n][1]
print(Dept,Office)
Page=df['Page'][(df['Office']==Office) & (df['Dept']==Dept)].tolist()[0]
Json=Clova(Year,Page)


水道局 会計課


In [123]:
[d['inferText'] for d in Json['fields']]

['課長',
 '百事',
 '技師',
 '業',
 '六九六',
 '主事',
 '孤上',
 '1',
 '事務員',
 '課長',
 '喝託員',
 '水道水源地氷川村日原氣象觀測事務喝託',
 '水神社奉祀福託',
 '水道水源地醫務署一年一一〇',
 '水道水源地ニ於ケル崩壞地ノ砂防施設關スル調査福託',
 '月 二 〇',
 '水道ニスス調調喝記',
 '水道用地林木ニ關スル調査事務喝託',
 '10',
 '上',
 '1',
 '下上',
 '務',
 '計',
 '課',
 '正人造 蔵',
 '合計利共加',
 '航支援器官',
 '課',
 '神田',
 '愨',
 '井',
 '知太',
 '久三',
 '宗太',
 '定',
 '郎一郎郎吉',
 '龜戸營業所長發',
 '充容充希',
 '五金、明天、五五台',
 '副牧入役西',
 'N',
 '上',
 'ト',
 'ト',
 '下',
 '正四、勤售等',
 '仲田聰治郎',
 '正五西',
 '熊',
 '七七八六人',
 '(税抜カ五九)',
 '世田谷',
 '、、、三八子',
 '六',
 '田千佐人',
 '磯 西 西 松 松',
 '木',
 '大條',
 '川武雄',
 '瀨戸',
 '深',
 '易一',
 '材料鐵板燒',
 '信',
 '竹北',
 '義俊靖龜',
 '雄一豪助',
 '吉一八郎',
 '萬 親',
 '覺']

In [33]:
json_object = json.dumps(dta, indent=4,
                        cls=NpEncoder)
save_path='C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\'
with open(save_path+"DataFrame.json", "w") as outfile:
    outfile.write(json_object)